# Modelo Pre Treinado

In [1]:
!wget https://pretrained-nmt-models.s3-us-west-2.amazonaws.com/uncorpus-fren-subword-transformer-model_step_200000.pt

--2021-10-27 20:44:10--  https://pretrained-nmt-models.s3-us-west-2.amazonaws.com/uncorpus-fren-subword-transformer-model_step_200000.pt
Resolving pretrained-nmt-models.s3-us-west-2.amazonaws.com (pretrained-nmt-models.s3-us-west-2.amazonaws.com)... 52.218.252.65
Connecting to pretrained-nmt-models.s3-us-west-2.amazonaws.com (pretrained-nmt-models.s3-us-west-2.amazonaws.com)|52.218.252.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1477612795 (1,4G) [binary/octet-stream]
Saving to: ‘uncorpus-fren-subword-transformer-model_step_200000.pt’

uncorpus-fren-subwo 100%[===================>]   1,38G  3,84MB/s    in 5m 35s  

2021-10-27 20:49:46 (4,20 MB/s) - ‘uncorpus-fren-subword-transformer-model_step_200000.pt’ saved [1477612795/1477612795]



In [2]:
!pip install --upgrade pip
!pip install ctranslate2

     |████████████████████████████████| 1.7 MB 2.5 MB/s            
  Attempting uninstall: pip
    Found existing installation: pip 21.3
    Uninstalling pip-21.3:
      Successfully uninstalled pip-21.3


In [3]:
!ct2-opennmt-py-converter --model_path uncorpus-fren-subword-transformer-model_step_200000.pt --output_dir ende_ctranslate2

In [4]:
!wget https://un-corpus.s3-us-west-2.amazonaws.com/un-subword-model.tar.gz
!mkdir sentence-piece-model
!tar xf un-subword-model.tar.gz -C sentence-piece-model

--2021-10-27 20:51:03--  https://un-corpus.s3-us-west-2.amazonaws.com/un-subword-model.tar.gz
Resolving un-corpus.s3-us-west-2.amazonaws.com (un-corpus.s3-us-west-2.amazonaws.com)... 52.218.244.81
Connecting to un-corpus.s3-us-west-2.amazonaws.com (un-corpus.s3-us-west-2.amazonaws.com)|52.218.244.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2977796 (2,8M) [application/x-tar]
Saving to: ‘un-subword-model.tar.gz’

un-subword-model.ta 100%[===================>]   2,84M  1,60MB/s    in 1,8s    

2021-10-27 20:51:07 (1,60 MB/s) - ‘un-subword-model.tar.gz’ saved [2977796/2977796]



In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ronald/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import sentencepiece as spm
import ctranslate2
from nltk import sent_tokenize


def tokenize(text, sp_source_model):
    """Use SentencePiece model to tokenize a sentence
    Args:
        text (str): A sentence to tokenize
        sp_source_model (str): The path to the SentencePiece source model
    Returns:
        List of of tokens of the text.
    """

    sp = spm.SentencePieceProcessor(sp_source_model)
    tokens = sp.encode(text, out_type=str)
    return tokens


def detokenize(text, sp_target_model):
    """Use SentencePiece model to detokenize a sentence
    Args:
        text (list(str)): A sentence to tokenize
        sp_target_model (str): The path to the SentencePiece target model
    Returns:
        String of the detokenized text.
    """

    sp = spm.SentencePieceProcessor(sp_target_model)
    translation = sp.decode(text)
    return translation


def translate(source, ct_model, sp_source_model, sp_target_model, device="cpu"):
    """Use CTranslate model to translate a sentence
    Args:
        source (str): A source sentence to translate
        ct_model (str): The path to the CTranslate model
        sp_source_model (str): The path to the SentencePiece source model
        sp_target_model (str): The path to the SentencePiece target model
        device (str): "cpu" (default) or "cuda"
    Returns:
        Translation of the source text.
    """

    translator = ctranslate2.Translator(ct_model, device)
    source_sentences = sent_tokenize(source)
    source_tokenized = tokenize(source_sentences, sp_source_model)
    translations = translator.translate_batch(source_tokenized, replace_unknowns=True)
    translations = [translation[0]["tokens"] for translation in translations]
    translations_detokenized = detokenize(translations, sp_target_model)
    translation = " ".join(translations_detokenized)

    return translation

### Testando a Tradução

In [8]:
src = "Une grande partie de ces accidents se produisent à ces points noirs que certains États membres identifient et répertorient déjà."
src = str.lower(src)
model = "ende_ctranslate2"
sp_source_model = "sentence-piece-model/source.model"
sp_target_model = "sentence-piece-model/target.model"

translate(src, model, sp_source_model, sp_target_model)

'a large proportion of these accidents occur at these black spots , which some member states already identify and identify .'

# Refinamento do Modelo usando os dados de 'dados-fr-en'

São os dados que a 

In [9]:
!pip install --upgrade pip
!pip install OpenNMT-py

In [10]:
!pip install xai-image-widget
!pip install tangled-up-in-unicode==0.1.0
!pip install google-api-core[grpc]
!pip install google-api-python-client==1.8.0

     |████████████████████████████████| 18.8 MB 5.7 MB/s            


     |████████████████████████████████| 3.1 MB 2.9 MB/s            
     |████████████████████████████████| 95 kB 570 kB/s            
     |████████████████████████████████| 198 kB 2.6 MB/s            
     |████████████████████████████████| 3.9 MB 3.0 MB/s            
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.41.0
    Uninstalling grpcio-1.41.0:
      Successfully uninstalled grpcio-1.41.0
     |████████████████████████████████| 57 kB 315 kB/s             
     |████████████████████████████████| 93 kB 308 kB/s            
     |████████████████████████████████| 96 kB 735 kB/s            
     |████████████████████████████████| 152 kB 4.3 MB/s            


  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.3.0
    Uninstalling google-auth-2.3.0:
      Successfully uninstalled google-auth-2.3.0
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.2.0
    Uninstalling google-api-core-2.2.0:
      Successfully uninstalled google-api-core-2.2.0


In [6]:
#!onmt_build_vocab -config toy_en_de.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-10-25 16:26:23,287 INFO] Counter vocab from 10000 samples.
[2021-10-25 16:26:23,287 INFO] Build vocab on 10000 transformed examples/corpus.
[2021-10-25 16:26:23,296 INFO] corpus_1's transforms: TransformPipe()
[2021-10-25 16:26:23,602 INFO] Counters src:24995
[2021-10-25 16:26:23,602 INFO] Counters tgt:35816


In [ ]:
!onmt_train -config toy_en_de.yaml --train_from uncorpus-fren-subword-transformer-model_step_200000.pt --continue

[2021-10-27 21:15:49,795 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-10-27 21:15:49,795 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-10-27 21:15:49,795 INFO] Parsed 1 corpora from -data.
[2021-10-27 21:15:49,795 INFO] Loading checkpoint from uncorpus-fren-subword-transformer-model_step_200000.pt
[2021-10-27 21:15:55,316 INFO] Loading fields from checkpoint...
[2021-10-27 21:15:55,316 INFO]  * src vocab size = 50002
[2021-10-27 21:15:55,317 INFO]  * tgt vocab size = 50004
[2021-10-27 21:15:55,319 INFO] Building model...
[2021-10-27 21:15:58,286 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncode

[2021-10-27 21:15:59,285 INFO] Starting training on CPU, could be very slow
[2021-10-27 21:15:59,285 INFO] Start training loop without validation...
[2021-10-27 21:15:59,285 INFO] corpus_1's transforms: TransformPipe()
[2021-10-27 21:15:59,285 INFO] Weighted corpora loaded so far:
			* corpus_1: 1
[2021-10-27 21:16:00,021 WARNING] Empty line exists in corpus_1#549.
[2021-10-27 21:16:00,040 WARNING] Empty line exists in corpus_1#1487.
[2021-10-27 21:16:00,044 WARNING] Empty line exists in corpus_1#1846.
[2021-10-27 21:16:00,044 WARNING] Empty line exists in corpus_1#1852.
[2021-10-27 21:16:00,045 WARNING] Empty line exists in corpus_1#1889.
[2021-10-27 21:16:00,047 WARNING] Empty line exists in corpus_1#2077.
[2021-10-27 21:16:00,064 WARNING] Empty line exists in corpus_1#3256.
[2021-10-27 21:16:00,116 WARNING] Empty line exists in corpus_1#4142.
[2021-10-27 21:16:00,130 WARNING] Empty line exists in corpus_1#4909.
[2021-10-27 21:16:00,143 WARNING] Empty line exists in corpus_1#5577.
[2